In [4]:
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from pathlib import PurePath
from shutil import copy

In [5]:
window = 256
fileloc = '/scratch/sk7898/pedbike/window_256'
data_type = 'upstream'
if data_type == 'upstream':
    filestrs = ['upstream_stft/Human/human_cuts_stft', 'upstream_stft/Nonhuman/non_human_cuts_stft']
    add_humans = None
    add_non_humans = ['downstream/Bike/bike_radial_cuts_stft']    
    classes = ['Human', 'Nonhuman']
    val_split = 0.2
    add_fraction = 0.1

In [6]:
outdir = fileloc
path_prefix = 'upstream_stft'
filenames = []
data = []
labels = []
seqs = []

[[filenames.append(os.path.join(os.path.join(fileloc, filestr), filename))\
  for filename in os.listdir(os.path.join(fileloc, filestr))] for filestr in filestrs]

[data.append(np.fromfile(open(filename, "r"), dtype=np.float64).reshape(-1, window*2)) for filename in filenames]
[labels.append(int(fname.split('_')[-1][0])) for fname in filenames]
[seqs.append(int(fname.split('_')[-3])) for fname in filenames]

if add_humans:
    human_filenames = []
    human_data = []
    human_labels = []
    human_seqs = []
    
    [[human_filenames.append(os.path.join(os.path.join(fileloc, filestr), filename))\
      for filename in os.listdir(os.path.join(fileloc, filestr))] for filestr in add_humans]
    
    human_indices = np.arange(len(human_filenames))
    human_indices = random.sample(human_indices.tolist(), k = int(add_fraction*len(human_filenames)))
    human_filenames = [human_filenames[i] for i in human_indices]
    
    [human_data.append(np.fromfile(open(filename, "r"), dtype=np.float64).reshape(-1, window*2))
      for filename in human_filenames]
    [human_labels.append(0) for fname in human_filenames]
    [human_seqs.append(int(fname.split('_')[-3])) for fname in human_filenames]
    
    filenames += human_filenames
    data += human_data
    labels += human_labels
    seqs += human_seqs
    
if add_non_humans:
    non_human_filenames = []
    non_human_data = []
    non_human_labels = []
    non_human_seqs = []
    
    [[non_human_filenames.append(os.path.join(os.path.join(fileloc, filestr), filename))\
      for filename in os.listdir(os.path.join(fileloc, filestr))] for filestr in add_non_humans]
    
    non_human_indices = np.arange(len(non_human_filenames))
    non_human_indices = random.sample(non_human_indices.tolist(), k = int(add_fraction*len(non_human_filenames)))
    non_human_filenames = [non_human_filenames[i] for i in non_human_indices]
    
    [non_human_data.append(np.fromfile(open(filename, "r"), dtype=np.float64).reshape(-1, window*2))\
     for filename in non_human_filenames]
    [non_human_labels.append(1) for fname in non_human_filenames]
    [non_human_seqs.append(int(fname.split('_')[-3])) for fname in non_human_filenames]
    
    filenames += non_human_filenames
    data += non_human_data
    labels += non_human_labels
    seqs += non_human_seqs

data = np.array(data)
labels = np.array(labels)
seqs = np.array(seqs)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/sk7898/pedbike/window_256/downstream/Bike/bike_radial_cuts_stft'

In [21]:
indices = np.arange(len(filenames))
X_train, X_val, y_train, y_val, indices_train, indices_val, seqs_train, seqs_val = train_test_split(data, labels,\
                                                                                                    indices, seqs, \
                                                                                                    test_size=val_split,\
                                                                                                    random_state=42)

os.makedirs(outdir, exist_ok=True)
os.makedirs(os.path.join(outdir, path_prefix), exist_ok=True)

# Save train data
np.save(os.path.join(outdir, path_prefix, "train.npy"), X_train)
np.save(os.path.join(outdir, path_prefix, "train_seqs.npy"), seqs_train)
np.save(os.path.join(outdir, path_prefix, "train_lbls.npy"), y_train)

# Save validation data
np.save(os.path.join(outdir, path_prefix, "val.npy"), X_val)
np.save(os.path.join(outdir, path_prefix, "val_seqs.npy"), seqs_val)
np.save(os.path.join(outdir, path_prefix, "val_lbls.npy"), y_val)

In [24]:
#Save the stft in .data format
files_train = [filenames[i] for i in indices_train]
files_val = [filenames[i] for i in indices_val]

train_dir = os.path.join(outdir, path_prefix, "stft_train")
val_dir = os.path.join(outdir, path_prefix, "stft_val")

for cls in classes:
    os.makedirs(os.path.join(train_dir,cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir,cls), exist_ok=True)

for tr in files_train:
    cur_class = PurePath(tr).parent.name.split('_')[0]
    if cur_class != 'Human':
        cur_class = 'Nonhuman'
    copy(tr, os.path.join(train_dir,cur_class))

for val in files_val:
    cur_class = PurePath(val).parent.name.split('_')[0]
    if cur_class != 'Human':
        cur_class = 'Nonhuman'
    copy(val, os.path.join(val_dir, cur_class))

**Train** Humans: 482 Non_Humans: 503

**Validation** Humans: 128 Non_Humans: 116

In [12]:
#Rename the files
# fileloc = '/scratch/sk7898/pedbike/upstream'
# filestrs = ['human_cuts_stft', 'nonhuman_cuts_stft']
# classes = ['Human', 'Nonhuman']

# filenames = []
# [[filenames.append(os.path.join(os.path.join(fileloc, filestr), filename))\
#   for filename in os.listdir(os.path.join(fileloc, filestr))] for filestr in filestrs]

# os.makedirs(os.path.join(fileloc, 'Human_stft'), exist_ok=True)
# os.makedirs(os.path.join(fileloc, 'Nonhuman_stft'), exist_ok=True)

# for f in filenames:
#     cur_class = PurePath(f).parent.name.split('_')[0]
#     new_class = 'Human' if 'human' == cur_class else 'Nonhuman'
#     copy(f, os.path.join(fileloc, new_class+'_stft', os.path.basename(f).replace(cur_class, new_class)))